## Ajuste Fino de um Modelo de IA Generativa para Tarefas de Pergunta e Respostas.

Neste notebook, irei trabalhar com dados extraídos do site oficial da Secretaria de Estado de Desenvolvimento Social de Goiás, com o objetivo de ajustar um modelo pré-treinado para lidar com perguntas e respostas relacionadas aos programas e benefícios oferecidos pela secretária. Para se adequar com a nossa necessidade, o modelo passará por um processo de ajuste fino. Primeiramente, iremos realizar um ajuste fino (Fine Tune, em inglês) total, onde todas as camadas do modelo pré-treinado serão modificadas, e depois apresentaremos o método PEFT (Ajuste Fino Eficiente de Parâmetros) que gasta menos recursos computacionais, e modifica apenas algumas camadas do modelo, conseguindo assim que ele retenha o conhecimento que já tem e evite o esquecimento catastrófico. Ao final, avaliaremos a precisão dos  dois modelos ajustados e determinaremos sua adequação para as tarefas definidas.

##### Baixando bibliotecas necessárias:

- transformers : A bibilioteca Transformers pode ser entendida como uma ferramenta poderosa para trabalhar com Processamento de Linguagem Natural 
- Datasets :  A biblioteca Datasets, também desenvolvida pela Hugging Face, é uma coleção de conjuntos de dados prontos para uso em uma variedade de tarefas de processamento de linguagem natural (PLN) e aprendizado de máquina (ML).
- Evaluate: Com essa biblioteca você ganha acesso a dezenas de métodos de avaliação para diferentes modelos, o que fdacilita a acilita a avaliação de modelos de machine learning e datasets.


In [1]:
#baixando arquivos necessários

!pip install transformers datasets evaluate

In [2]:
#importando as funções importantes para o fione-tune
from transformers import AutoTokenizer, pipeline, AutoModelForPreTraining, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import json

##### Escolhendo o modelo
é importante escolher qual modelo pre treinaido você quer escolher.

iremos usar o bloom

In [3]:
#Escolhendo o modelo para realizar 
model_name = "bigscience/bloom-560m"
model = AutoModelForPreTraining.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

KeyboardInterrupt: 

##### Preparando os dados

Quando vamos realizar um Fine-Tune, antes de qualquer coisa, precisamos preparar os dados que serão utilizados para realizar o ajuste do modelo.

Os dados utilizados neste algoritmo foram criados com base nos beneficios oferecidos pela Secretaria de Estado de Desenvolvimento Social de Goiás. Essas informações podem ser consultadas neste [link]("https://goias.gov.br/social/#").

O arquivo na extensão .json que foi utilizado está disponivel para download neste [link]().

Extrai as informações do arquivo e atribui á um dicionario

Etiquetei cada prompt com o nome do programa correspondente para melhorar a identificação do programa que se refere cada pergunta

In [ ]:
dataset_por_programa = {} #criando um dicionario para receber as informações do arquivo json "relacionados3.json"

# Carregar o arquivo JSON
with open('relacionados3.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Percorrer cada programa no arquivo JSON
for programa, lista_perguntas_respostas in data.items():
    # Criar uma entrada para o programa no dicionário
    dataset_por_programa[programa] = []
    # Percorrer cada item (dicionário) etiquetar prompts e completions
    for item in lista_perguntas_respostas:
        dataset_por_programa[programa].append({'prompt': f"{programa} : {item['prompt']}", 'completion': item['completion']})


In [ ]:
print(dataset_por_programa)

Neste código, estamos iterando sobre o dicionário e extraindo as perguntas e respostas para cada programa. Em seguida, estruturamos essas perguntas e respostas em pares e os adicionamos à lista perguntas_respostas.

In [ ]:
perguntas_respostas = []

for programa, lista_perguntas in dataset_por_programa.items():
    for pergunta_resposta in lista_perguntas:
        pergunta = pergunta_resposta["prompt"]
        respostas = pergunta_resposta["completion"]
        for resposta in respostas:
            # Adiciona o par pergunta-resposta à lista
            perguntas_respostas.append((pergunta, resposta))

print(perguntas_respostas[:5])  # Exibe os primeiros 5 pares pergunta-resposta

### Testando fazer com csv

In [ ]:
!pip install pandas
import pandas as pd

In [ ]:
# Carregar o arquivo CSV
df = pd.read_csv('dataset.csv')

# Dividir o conjunto de dados em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Salvar os conjuntos de treinamento e teste em novos arquivos CSV
train_df.to_csv('treino.csv', index=False)
test_df.to_csv('teste.csv', index=False)

In [ ]:
data_file = {"train": "treino.csv", "test":'teste.csv'}
dataSetCompleto = load_dataset("csv", data_files=data_file)

#atribuindo a variavel dados de tewste e de treino

treino = dataSetCompleto['train']
teste = dataSetCompleto['test']

In [ ]:
print(treino)

In [ ]:
#Aqui criamos uma função para facilitar a tokenização

def tokenizar(dados):
    sequencia = dados["pergunta"] + " [SEP] " + dados["resposta"]
    return tokenizer(sequencia, truncation=True, padding="max_length", max_length=1000)

#fale sobre o mecanimo de atenção e sobre usar tokenização conjunta ou não

In [15]:
#Falar do que cada coisa faz --> map
dataset_teste = teste.map(tokenizar)
dataset_treino = treino.map(tokenizar)

Map:   0%|          | 0/531 [00:00<?, ? examples/s]

Map:   0%|          | 0/2120 [00:00<?, ? examples/s]

In [ ]:
# definir função para computar as metricas de analiação do modelo

def computar_metricas():
    

In [ ]:
#configurarndo parametros de treino

training_args = TrainingArguments(
    output_dir='./modeloAjustado',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
)